# Upload Images

Before we can apply pretrained models to classify images, we have to upload the images to the Google Colab environment.

The `upload_files` function below can be used to upload files.

In [ ]:
from google.colab import files

def upload_files():
    uploaded = files.upload()
    for k, v in uploaded.items():
        open(k, 'wb').write(v)
    return list(uploaded.keys())

Run the following code block to upload your image files. This function returns a list of image paths which will be stored in `files`.

In [ ]:
files = upload_files()

# Using Pre-trained Models

Initialize model with the best available weights.

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

Before using the pre-trained models, one must preprocess the image (resize with right resolution/interpolation, apply inference transforms, rescale the values etc). There is no standard way to do this as it depends on how a given model was trained. It can vary across model families, variants or even weight versions. Using the correct preprocessing method is critical and failing to do so may lead to decreased accuracy or incorrect outputs.

All the necessary information for the inference transforms of each pre-trained model is provided on its weights documentation. To simplify inference, TorchVision bundles the necessary preprocessing transforms into each model weight. These are accessible via the weight.transforms attribute:

In [ ]:
import torch
from torchvision.io import read_image

# Load images from paths into tensors
imgs = []
for fp in files:
    img = read_image(fp)
    imgs.append(img)

# Initialize the inference transforms
preprocess = weights.transforms()

# Apply inference preprocessing transforms
if len(imgs) == 1:
    # Add a batch dimension after preprocessing
    batch = preprocess(imgs[0]).unsqueeze(0)
else:
    # Stack multiple preprocessed image
    batch = torch.stack([preprocess(img) for img in imgs])

print(batch.shape)

Use the model and print the predicted category.

In [ ]:
# Prediction the probability of each class
logits = model(batch)
probas = logits.softmax(1)
preds = probas.argmax(1)

for i in range(len(preds)):
    class_id = preds[i].item()
    score = probas[i][class_id].item()
    category_name = weights.meta["categories"][class_id]
    print(f"{category_name}: {100 * score:.1f}%")

# Exercises: Change the model

You need to change the pre-trained model to be something else. Please have a look [here](https://pytorch.org/vision/stable/models.html#table-of-all-available-classification-weights) and pick one model of your choice. Then write the code below to apply a different model.

In [ ]:
# YOUR CODE HERE